In [1]:
import cv2

In [2]:
cap=cv2.VideoCapture(r'C:\Users\Aditya\Desktop\object_tracking\object_tracking\highway2.mp4')

In [3]:
from tracker import *

In [4]:
tracker = EuclideanDistTracker()

In [5]:
object_detector=cv2.createBackgroundSubtractorMOG2(history=100 ,varThreshold=40) 

import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids





### object detection

In [6]:
while True:
    ret, frame= cap.read() 
    height , width, _ = frame.shape
    print(height ,width)
    

    #extract region of interest
    roi = frame[340:700 , 500:790]

    
    #apply masking #object detection
    mask= object_detector.apply(roi)
    _,mask= cv2.threshold(mask , 254 , 255 , cv2.THRESH_BINARY)  #keep between 254&255 / below 244: cut it out )
    contour,_= cv2.findContours(mask , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections=[]



    
    for cnt in contour:
        #lets remove small/unwanted contours
        area=cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi , [cnt] , -1 , (0,255,0) , 2)
            x,y,w,h = cv2.boundingRect(cnt)
            #cv2.rectangle(roi , (x,y) , (x+w , y+h), (0,255,0), 2)
            detections.append([x,y,w,h])

        #object tracking 
    boxes_id= tracker.update(detections)
    for box_id in boxes_id:
        x,y,w,h,id=box_id
        cv2.putText(roi , str(id), (x,y-15) , cv2.FONT_HERSHEY_PLAIN ,1,(255,0,0), 2)
        cv2.rectangle(roi, (x,y), (x+w , y+h), (0,255,0), 3)


    
        
    
    cv2.imshow('roi', roi)
    cv2.imshow("frame", frame)
    cv2.imshow('Mask', mask)

    if cv2.waitKey(40) & 0xFF == ord('q'):
         break

cap.release()
cv2.destroyAllWindows()





720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
720 1280
{0: (103, 22)}
720 1280
{0: (103, 26)}
720 1280
{0: (103, 30)}
720 1280
{0: (101, 34)}
720 1280
{0: (100, 38)}
720 1280
{0: (100, 42)}
720 1280
{0: (100, 47)}
720 1280
{0: (99, 51)}
720 1280
{0: (99, 57)}
720 1280
{0: (98, 61)}
720 1280
{0: (98, 66)}
720 1280
{0: (97, 71)}
720 1280
{0: (95, 76)}
720 1280
{0: (94, 83)}
720 1280
{0: (94, 87)}
720 1280
{0: (94, 90)}
720 1280
{0: (93, 100)}
720 1280
{0: (92, 107)}
720 1280
{0: (92, 114)}
720 1280
{0: (89, 120)}
720 1280
{0: (89, 127)}
720 1280
{0: (89, 134)}
720 1280
{0: (85, 142)}
720 1280
{0: (85, 144)}
720 1280
{0: (83, 153), 1: (186, 9)}
{0: (83, 153), 1: (185, 11)}
720 1280
{0: (84, 160), 1: (185, 11)}
{0: (84, 160), 1: (186, 12)}
720 1280
{0: (80, 169), 1: (186, 12)}
{0: (80, 169), 1: (186, 14)}
720 1280

## 